### Import Required Libraries and Set Up Environment Variables

In [12]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
from pandas import json_normalize
import json
import re

In [13]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [15]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
nyt_query = f"{url}api-key={nyt_api_key}&sort={sort}&fq={filter_query}&facet_fields={field_list}&begin_date={begin_date}&end_date={end_date}"

In [11]:
# Create an empty list to store the reviews
reviews_list = []

# loop through pages 0-19
for page in range(0, 20):
    print(f"Getting results for page {page}")

    response = requests.get(f"{nyt_query}&page={page}").json()

    articles = response['response']['docs']
    if len(articles) == 0:
        print(f"No results for page {page}")
        break

    reviews_list.extend(articles)
    time.sleep(12)

print(len(reviews_list))


Getting results for page 0
Getting results for page 1
Getting results for page 2
Getting results for page 3
Getting results for page 4
Getting results for page 5
Getting results for page 6
Getting results for page 7
Getting results for page 8
Getting results for page 9
Getting results for page 10
Getting results for page 11
Getting results for page 12
Getting results for page 13
Getting results for page 14
Getting results for page 15
Getting results for page 16
Getting results for page 17
Getting results for page 18
Getting results for page 19
200


In [18]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
first_five = reviews_list[:5]
json_output = json.dumps(first_five, indent=4)
print(json_output)

[
    {
        "abstract": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "lead_paragraph": "A trashy treat coated in a high-art gloss, \u201cThe Attachment Diaries\u201d gleefully kneads melodrama, noir, horror and sexual perversion into a pathological romance between two deeply damaged women.",
        "print_section": "C",
        "print_page": "7",
        "source": "The New York Times",
        "multimedia": [
            {
                "rank": 0,
                "subtype": "xlarge",
                "caption": null,
                "credit": null,
                "type": "image",
                "url": "images/2023/05/26/multimedia/attachment1-mbcw/attachment1-mbcw-articl

In [21]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
df_original = json_normalize(reviews_list)
df = df_original.copy()
df

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,Two childhood friends navigate cultural differ...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,A glossy lesson in how to pour nontraditional ...,C,8,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,Rachael Leigh Cook stars in this bland rom-com...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The first thing we learn about Amanda (Rachael...,C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,A radiant Virginie Efira stars as a Parisian t...,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,When a woman falls in love in the sensitive Fr...,C,4,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,...,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,This moody romance stars Tatiana Maslany (“Orp...,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,Acting prowess is in vast abundance in the moo...,C,9,The New York Times,"[{'rank': 0, 'subtype': 'hpLarge', 'caption': ...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,...,Review: A Combustible Pair Find Love in ‘The O...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None
196,A nurse travels to the Ottoman Empire on the e...,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,“The Ottoman Lieutenant” is an overwrought nur...,C,10,The New York Times,"[{'rank': 0, 'subtype': 'articleInline', 'capt...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,...,"Review: 

In [22]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019.
# End string should include " Review" to avoid cutting title early
pattern = r'\u2018(.*?)\u2019'

df['title'] = df['headline.main'].str.extract(pattern)
df['title'] = df['title'].str.strip()#+ " Review"
df


,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,...,None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,Two childhood friends navigate cultural differ...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,A glossy lesson in how to pour nontraditional ...,C,8,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What
2,Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,Rachael Leigh Cook stars in this bland rom-com...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The first thing we learn about Amanda (Rachael...,C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist
4,A radiant Virginie Efira stars as a Parisian t...,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,When a woman falls in love in the sensitive Fr...,C,4,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,...,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,This moody romance stars Tatiana Maslany (“Orp...,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,Acting prowess is in vast abundance in the moo...,C,9,The New York Times,"[{'rank': 0, 'subtype': 'hpLarge', 'caption': ...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:54:58+0000,...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,A nurse travels to the Ottoman Empire on the e...,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,“The Ottoman Lieutenant” is an overwrought nur...,C,10,The New York Times,"[{'rank': 0, 'subtype': 'articleInline', 'capt...","[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-03-09T21:53:12+0000,...,None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,Josh Kornbluth runs afoul of the Internal R

In [23]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    if not isinstance(keyword_list, list):
        return keyword_list

    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};"
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
df['keywords'] = df['keywords'].apply(extract_keywords)
df

,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,...,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,...,None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,Two childhood friends navigate cultural differ...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,A glossy lesson in how to pour nontraditional ...,C,8,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What
2,Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,Rachael Leigh Cook stars in this bland rom-com...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The first thing we learn about Amanda (Rachael...,C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist
4,A radiant Virginie Efira stars as a Parisian t...,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,When a woman falls in love in the sensitive Fr...,C,4,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,...,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,This moody romance stars Tatiana Maslany (“Orp...,https://www.nytimes.com/2017/03/09/movies/the-...,This moody romance stars Tatiana Maslany (“Orp...,Acting prowess is in vast abundance in the moo...,C,9,The New York Times,"[{'rank': 0, 'subtype': 'hpLarge', 'caption': ...",subject: Movies;creative_works: The Other Half...,2017-03-09T21:54:58+0000,...,None,None,Review: A Combustible Pair Find Love in ‘The O...,None,None,None,By Andy Webster,"[{'firstname': 'Andy', 'middlename': None, 'la...",None,The Other Half
196,A nurse travels to the Ottoman Empire on the e...,https://www.nytimes.com/2017/03/09/movies/revi...,A nurse travels to the Ottoman Empire on the e...,“The Ottoman Lieutenant” is an overwrought nur...,C,10,The New York Times,"[{'rank': 0, 'subtype': 'articleInline', 'capt...",subject: Movies;creative_works: The Ottoman Li...,2017-03-09T21:53:12+0000,...,None,None,"Review: Love as the World Wars, in ‘The Ottoma...",None,None,None,By Neil Genzlinger,"[{'firstname': 'Neil', 'middlename': None, 'la...",None,The Ottoman Lieutenant
197,Josh Kornbluth runs afoul of the Internal Reve...,htt

In [24]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = df['title'].to_list()
title_list

['The Attachment Diaries',
 'What',
 'You Can Live Forever',
 'A Tourist',
 'Other People',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Cloud',
 'A Journal for Jordan',
 'West Side Story',
 'Aulcie',
 'Love Is Love 

### Access The Movie Database API

In [25]:
# Prepare The Movie Database query
tmdb_search_url = "https://api.themoviedb.org/3/search/movie"
api_key_query = f"?api_key={tmdb_api_key}"
tmdb_query = f"{tmdb_search_url}{api_key_query}"

In [53]:
tmdb_movies_list = []
count = 1
for title in title_list:
    if count % 50 == 0:
        print(f"sleeping at {count} requests")
        time.sleep(1)

    movie_id = ""
    response = requests.get(f"{tmdb_query}&query={title}").json()
    try:
        movie_id = response["results"][0]["id"]
        tmdb_movie_url = "https://api.themoviedb.org/3/movie"
        query_url = f"{tmdb_movie_url}/{movie_id}{api_key_query}"
        response = requests.get(query_url).json()
        genres = []
        spoken_languages = []
        production_countries = []

        for genre in response["genres"]:
            genres.append(genre["name"])

        for language in response["spoken_languages"]:
            spoken_languages.append(language["name"])

        for country in response["production_countries"]:
            production_countries.append(country["name"])

        movie_info = {
            "title": title,
            "original_title": response["original_title"],
            "budget": response["budget"],
            "genre": genres,
            "language": response["original_title"],
            "spoken_languages": spoken_languages,
            "homepage": response["homepage"],
            "overview": response["overview"],
            "popularity": response["popularity"],
            "runtime": response["runtime"],
            "revenue": response["revenue"],
            "release_date": response["release_date"],
            "vote_average": response["vote_average"],
            "vote_count": response["vote_count"],
            "production_countries": production_countries,
        }

        tmdb_movies_list.append(movie_info)
        print(f"Movie '{title}' was found")
        count += 1
    except Exception as e:
        RED = "\033[31m"
        RESET = "\033[0m"
        print(f"\n{RED}Error when looking up '{title}'. Movie ID '{movie_id}': {e}{RESET}\n")

print("tmdb_movies_list length:", len(tmdb_movies_list))


Movie 'The Attachment Diaries' was found
Movie 'What' was found
Movie 'You Can Live Forever' was found
Movie 'A Tourist' was found
Movie 'Other People' was found
Movie 'One True Loves' was found
Movie 'The Lost Weekend: A Love Story' was found
Movie 'A Thousand and One' was found
Movie 'Your Place or Mine' was found
Movie 'Love in the Time of Fentanyl' was found
Movie 'Pamela, a Love Story' was found
Movie 'In From the Side' was found
Movie 'After Love' was found
Movie 'Alcarràs' was found
Movie 'Nelly & Nadine' was found
Movie 'Lady Chatterley' was found
Movie 'The Sound of Christmas' was found
Movie 'The Inspection' was found
Movie 'Bones and All' was found
Movie 'My Policeman' was found
Movie 'About Fate' was found
Movie 'Waiting for Bojangles' was found
Movie 'I Love My Dad' was found
Movie 'A Love Song' was found
Movie 'Alone Together' was found
Movie 'Art of Love' was found
Movie 'The Wheel' was found
Movie 'Thor: Love and Thunder' was found
Movie 'Both Sides of the Blade' was fo

In [54]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))

[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "genre": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "language": "El apego",
        "spoken_languages": [
            "Espa\u00f1ol"
        ],
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 2.232,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "production_countries": [
            "Argentina"
        ]


In [55]:
# Convert the results to a DataFrame
df_movies_original = json_normalize(tmdb_movies_list)
df_movies = df_movies_original.copy()
df_movies

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",El apego,[Español],,"Argentina, 1970s. A desperate young woman goes...",2.232,102,0,2021-10-07,3.000,4,[Argentina]
1,What,What...,0,[],What...,[],,,0.939,0,0,,0.000,0,[]
2,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",You Can Live Forever,"[English, Français]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",41.249,96,15055,2023-03-24,6.474,38,"[Canada, United States of America]"
3,A Tourist,A Tourist's Guide to Love,0,"[Romance, Comedy]",A Tourist's Guide to Love,"[English, Tiếng Việt]",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",27.389,96,0,2023-04-21,6.288,165,[United States of America]
4,Other People,Other People,0,"[Comedy, Drama]",Other People,[English],,"David, a struggling comedy writer fresh off fr...",6.032,97,93000,2016-09-09,6.300,193,[United States of America]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194,The Other Half,The Other Half,0,"[Drama, Romance]",The Other Half,[English],http://theotherhalfmovie.com/,A grief-stricken man and a bipolar woman fall ...,28.008,103,0,2016-12-02,6.300,24,[Canada]
195,The Ottoman Lieutenant,The Ottoman Lieutenant,40000000,"[Romance, Drama, War]",The Ottoman Lieutenant,"[Türkçe, English]",https://www.uphe.com/movies/the-ottoman-lieute...,"Lillie, a determined American woman, ventures ...",31.988,111,413844,2017-03-10,6.164,247,"[Turkey, United States of America]"
196,Love & Taxes,Romantik Hırsız,0,"[Romance, Action, Drama]",Romantik Hırsız,[Türkçe],https://www.netflix.com/title/81689799,After learning that the art thief she has been...,56.059,99,0,2024-03-13,6.457,115,[Turkey]
197,"Everybody Loves Somebody,",Everybody Loves Somebody,2000000,"[Romance, Comedy]",Everybody Loves Somebody,"[English, Español]",,"On the surface, Clara Barron seems to have it ...",16.079,102,0,2017-02-10,6.538,159,[Mexico]


### Merge and Clean the Data for Export

In [131]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df_original = pd.merge(df_movies, df, on='title', how='inner')
merged_df = merged_df_original.copy()
merged_df

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,document_type,news_desk,section_name,type_of_material,_id,word_count,uri,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"[Drama, Mystery, Thriller, Horror]",El apego,[Español],,"Argentina, 1970s. A desperate young woman goes...",2.232,102,0,2021-10-07,3.000,4,[Argentina],A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,article,Weekend,Movies,Review,nyt://article/428e3586-a19f-5912-9f7c-6b913068...,295,nyt://article/428e3586-a19f-5912-9f7c-6b913068...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,What,What...,0,[],What...,[],,,0.939,0,0,,0.000,0,[],Two childhood friends navigate cultural differ...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,A glossy lesson in how to pour nontraditional ...,C,8,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,article,Weekend,Movies,Review,nyt://article/7060f9b8-af27-5673-abf1-68f4e0e8...,287,nyt://article/7060f9b8-af27-5673-abf1-68f4e0e8...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,You Can Live Forever,You Can Live Forever,0,"[Drama, Romance]",You Can Live Forever,"[English, Français]",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",41.249,96,15055,2023-03-24,6.474,38,"[Canada, United States of America]",Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,article,Weekend,Movies,Review,nyt://article/01d68af6-5e5d-52c9-bf5a-87e068ac...,294,nyt://article/01d68af6-5e5d-52c9-bf5a-87e068ac...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,A Tourist,A Tourist's Guide to Love,0,"[Romance, Comedy]",A Tourist's Guide to Love,"[English, Tiếng Việt]",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",27.389,96,0,2023-04-21,6.288,165,[United States of America],Rachael Leigh Cook stars in this bland rom-com...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The first thing we learn about Amanda (Rachael...,C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,article,Weekend,Movies,Review,nyt://article/df6f1c8d-527e-5bd6-81c8-24708f91...,276,nyt://article/df6f1c8d-527e-5bd6-81c8-24708f91...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,Other People,Other People,0,"[Comedy, Dram

In [132]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
list_columns = ["genre", "spoken_languages", "production_countries"]
characters_to_remove = ['[', ']', "'", '"']

for col in list_columns:
    merged_df[col] = merged_df[col].astype(str)
    # merged_df[col] = merged_df[col].apply(lambda x: ', '.join(str(item) for item in x) if isinstance(x, list) else str(x))
    for char in characters_to_remove:
        merged_df[col] = merged_df[col].str.replace(char, '', regex=False)


pd.set_option('display.max_columns', None)
merged_df.head()

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,multimedia,keywords,pub_date,document_type,news_desk,section_name,type_of_material,_id,word_count,uri,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",El apego,Español,,"Argentina, 1970s. A desperate young woman goes...",2.232,102,0,2021-10-07,3.000,4,Argentina,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,article,Weekend,Movies,Review,nyt://article/428e3586-a19f-5912-9f7c-6b913068...,295,nyt://article/428e3586-a19f-5912-9f7c-6b913068...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,What,What...,0,,What...,,,,0.939,0,0,,0.000,0,,Two childhood friends navigate cultural differ...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,A glossy lesson in how to pour nontraditional ...,C,8,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...","subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,article,Weekend,Movies,Review,nyt://article/7060f9b8-af27-5673-abf1-68f4e0e8...,287,nyt://article/7060f9b8-af27-5673-abf1-68f4e0e8...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",You Can Live Forever,"English, Français",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",41.249,96,15055,2023-03-24,6.474,38,"Canada, United States of America",Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,article,Weekend,Movies,Review,nyt://article/01d68af6-5e5d-52c9-bf5a-87e068ac...,294,nyt://article/01d68af6-5e5d-52c9-bf5a-87e068ac...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,A Tourist,A Tourist's Guide to Love,0,"Romance, Comedy",A Tourist's Guide to Love,"English, Tiếng Việt",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",27.389,96,0,2023-04-21,6.288,165,United States of America,Rachael Leigh Cook stars in this bland rom-com...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The first thing we learn about Amanda (Rachael...,C,7,The New York Times,"[{'rank': 0, 'subtype': 'xlarge', 'caption': N...",subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,article,Weekend,Movies,Review,nyt://article/df6f1c8d-527e-5bd6-81c8-24708f91...,276,nyt://article/df6f1c8d-527e-5bd6-81c8-24708f91...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,Other People,Other People,0,"Comedy, Drama",Other People,English,,

In [133]:
# Drop "byline.person" column
if 'byline.person' in merged_df.columns:
    merged_df = merged_df.drop(columns=['byline.person'])
# dropping multimedia because this is an array of objects like byline.person
if 'multimedia' in merged_df.columns:
    merged_df = merged_df.drop(columns=['multimedia'])
merged_df.head()

,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,keywords,pub_date,document_type,news_desk,section_name,type_of_material,_id,word_count,uri,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",El apego,Español,,"Argentina, 1970s. A desperate young woman goes...",2.232,102,0,2021-10-07,3.000,4,Argentina,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,article,Weekend,Movies,Review,nyt://article/428e3586-a19f-5912-9f7c-6b913068...,295,nyt://article/428e3586-a19f-5912-9f7c-6b913068...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,What,What...,0,,What...,,,,0.939,0,0,,0.000,0,,Two childhood friends navigate cultural differ...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,A glossy lesson in how to pour nontraditional ...,C,8,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,article,Weekend,Movies,Review,nyt://article/7060f9b8-af27-5673-abf1-68f4e0e8...,287,nyt://article/7060f9b8-af27-5673-abf1-68f4e0e8...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None
2,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",You Can Live Forever,"English, Français",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",41.249,96,15055,2023-03-24,6.474,38,"Canada, United States of America",Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,article,Weekend,Movies,Review,nyt://article/01d68af6-5e5d-52c9-bf5a-87e068ac...,294,nyt://article/01d68af6-5e5d-52c9-bf5a-87e068ac...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
3,A Tourist,A Tourist's Guide to Love,0,"Romance, Comedy",A Tourist's Guide to Love,"English, Tiếng Việt",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",27.389,96,0,2023-04-21,6.288,165,United States of America,Rachael Leigh Cook stars in this bland rom-com...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The first thing we learn about Amanda (Rachael...,C,7,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,article,Weekend,Movies,Review,nyt://article/df6f1c8d-527e-5bd6-81c8-24708f91...,276,nyt://article/df6f1c8d-527e-5bd6-81c8-24708f91...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None
4,Other People,Other People,0,"Comedy, Drama",Other People,English,,"David, a struggling comedy writer fresh off fr...",6.032,97,93000,2016-09-09,6.300,193,United States of America,A radiant Virginie Efira stars as a Parisian t...,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,When a woman falls in love in the sensitive Fr...,C,4,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,article,Weekend,Movies,Revie

In [134]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates().reset_index(drop=True)
merged_df


,title,original_title,budget,genre,language,spoken_languages,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,production_countries,abstract,web_url,snippet,lead_paragraph,print_section,print_page,source,keywords,pub_date,document_type,news_desk,section_name,type_of_material,_id,word_count,uri,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.organization
0,The Attachment Diaries,El apego,0,"Drama, Mystery, Thriller, Horror",El apego,Español,,"Argentina, 1970s. A desperate young woman goes...",2.232,102,0,2021-10-07,3.000,4,Argentina,A gynecologist and her patient form a horrifyi...,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,"A trashy treat coated in a high-art gloss, “Th...",C,7,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,article,Weekend,Movies,Review,nyt://article/428e3586-a19f-5912-9f7c-6b913068...,295,nyt://article/428e3586-a19f-5912-9f7c-6b913068...,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,None
1,What,What...,0,,What...,,,,0.939,0,0,,0.000,0,,Two childhood friends navigate cultural differ...,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,A glossy lesson in how to pour nontraditional ...,C,8,The New York Times,"subject: Movies;persons: Kapur, Shekhar;person...",2023-05-04T17:16:45+0000,article,Weekend,Movies,Review,nyt://article/7060f9b8-af27-5673-abf1-68f4e0e8...,287,nyt://article/7060f9b8-af27-5673-abf1-68f4e0e8...,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,None
2,You Can Live Forever,You Can Live Forever,0,"Drama, Romance",You Can Live Forever,"English, Français",https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",41.249,96,15055,2023-03-24,6.474,38,"Canada, United States of America",Religion comes between two girls falling in lo...,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,"In “You Can Live Forever,” Jaime and Marike do...",C,9,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,article,Weekend,Movies,Review,nyt://article/01d68af6-5e5d-52c9-bf5a-87e068ac...,294,nyt://article/01d68af6-5e5d-52c9-bf5a-87e068ac...,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,None
3,A Tourist,A Tourist's Guide to Love,0,"Romance, Comedy",A Tourist's Guide to Love,"English, Tiếng Việt",https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",27.389,96,0,2023-04-21,6.288,165,United States of America,Rachael Leigh Cook stars in this bland rom-com...,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The first thing we learn about Amanda (Rachael...,C,7,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,article,Weekend,Movies,Review,nyt://article/df6f1c8d-527e-5bd6-81c8-24708f91...,276,nyt://article/df6f1c8d-527e-5bd6-81c8-24708f91...,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,None
4,Other People,Other People,0,"Comedy, Drama",Other People,English,,"David, a struggling comedy writer fresh off fr...",6.032,97,93000,2016-09-09,6.300,193,United States of America,A radiant Virginie Efira stars as a Parisian t...,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,When a woman falls in love in the sensitive Fr...,C,4,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,article,Weekend,Movies,Revie

In [124]:
# Export data to CSV without the index
merged_df.to_csv('output.csv', index=False)